In [3]:
import tensorflow as tf
import os
import shutil
import random

In [11]:
# define source and destination paths
dataset_path=r"C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Face_mask_identifier\data"
train_path=r"C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Face_mask_identifier\train"
val_path=r"C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Face_mask_identifier\val"

# define categories
categories=['with_mask','without_mask']

# split ratio
split_ratio=0.8

In [12]:
for category in categories:
    os.makedirs(os.path.join(train_path,category),exist_ok=True)
    os.makedirs(os.path.join(val_path,category),exist_ok=True)
    
    # get all images in category folder
    images=os.listdir(os.path.join(dataset_path,category))
    random.shuffle(images)
    
    # split data
    train_size=int(len(images)*split_ratio)
    train_images=images[:train_size]
    val_images=images[train_size:]
    
    # move images to train annd val folders
    for img in train_images:
        shutil.copy(os.path.join(dataset_path,category,img),os.path.join(train_path,category,img))
    for img in val_images:
        shutil.copy(os.path.join(dataset_path,category,img),os.path.join(val_path,category,img))
print("Dataset successfully split into train and validation sets!")

Dataset successfully split into train and validation sets!


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image data generators(Augmentation)
train_datagen=ImageDataGenerator(rescale=1/255,rotation_range=20,horizontal_flip=True)
val_datagen=ImageDataGenerator(rescale=1/255)

# Load images
train_generator=train_datagen.flow_from_directory(train_path,target_size=(150,150),batch_size=32,class_mode="binary")
val_generator=val_datagen.flow_from_directory(val_path,target_size=(150,150),batch_size=32,class_mode="binary")

# define CNN model
model=Sequential([Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)),
                  MaxPooling2D(2,2),
                  Conv2D(64,(3,3),activation='relu'),
                  MaxPooling2D(2,2),
                  Conv2D(128,(3,3),activation='relu'),
                  MaxPooling2D(2,2),
                  Flatten(),
                  Dense(512,activation='relu'),
                  Dropout(0.5),
                  Dense(1,activation='sigmoid') # binary classification (cats vs dogs)
                  ])
# compile model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# train  model
model.fit(train_generator,validation_data=val_generator,epochs=5)

# save model
model.save("facemask_identifier.h5")
print("model training completed and saved!")

Found 6042 images belonging to 2 classes.
Found 1511 images belonging to 2 classes.


c:\Users\hexdy\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\hexdy\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
128/189 ━━━━━━━━━━━━━━━━━━━━ 31s 519ms/step - accuracy: 0.7039 - loss: 0.6611

c:\Users\hexdy\anaconda3\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


189/189 ━━━━━━━━━━━━━━━━━━━━ 109s 569ms/step - accuracy: 0.7429 - loss: 0.5788 - val_accuracy: 0.8882 - val_loss: 0.2863
Epoch 2/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 75s 394ms/step - accuracy: 0.8934 - loss: 0.2654 - val_accuracy: 0.8795 - val_loss: 0.2752
Epoch 3/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 75s 394ms/step - accuracy: 0.9127 - loss: 0.2171 - val_accuracy: 0.9285 - val_loss: 0.1976
Epoch 4/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 74s 390ms/step - accuracy: 0.9180 - loss: 0.2003 - val_accuracy: 0.9246 - val_loss: 0.1880
Epoch 5/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 75s 397ms/step - accuracy: 0.9292 - loss: 0.1854 - val_accuracy: 0.9365 - val_loss: 0.1568


model training completed and saved!


In [23]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Load trained model
model = tf.keras.models.load_model("facemask_identifier.h5")

# Load and preprocess test image
img_path = r"C:\Users\hexdy\OneDrive\Desktop\Batch_AI\Face_mask_identifier\sample_images_for_pred\masked.jpg"
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict
prediction = model.predict(img_array)
prediction

if prediction > 0.5:
    print("The image is without MASK")
else:
    print("The image is with MASK")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
The image is with MASK


In [14]:
train_generator.class_indices

{'with_mask': 0, 'without_mask': 1}